In [3]:
import os

In [4]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [6]:
# 만약에 파일 없으면 !mkdir "KSLProject"

# %cd 'MyDrive'
# %cd 'KSLProject'
# %cd 'hmdb51'

/gdrive/MyDrive/KSLProject


In [ ]:
!wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar

--2021-05-19 18:22:59--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2021-05-19 18:22:59--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: ‘hmdb51_org.rar’

hmdb51_org.rar      100%[===================>]   1.98G  19.8MB/s    in 74s     

2021-05-19 18:24:13 (27.4 MB/s) - ‘hmdb51_org.rar’ saved [2124008126/2124008126]



In [ ]:
# !mkdir 'hmdb51'
%cd 'hmdb51'

/gdrive/My Drive/KSLProject/hmdb51


In [ ]:
#unzip rar file
!unrar x ../hmdb51_org.rar

In [ ]:
ls

In [ ]:
ls

In [ ]:
!unrar x for i in brush_hair.rar

In [ ]:
file_list = []
file_list = os.listdir(os.getcwd())
print(file_list)

In [ ]:
cur_path = os.getcwd() 
cur_path

In [ ]:
# for x in file_list:
#   !unrar x cur_path
!unrar x for i in file_list

In [9]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [10]:
frames = 15
Width = 256
Height = 256

In [11]:
#비디오명과 레이블 짝을 가져온다.
def load_video_names(path):

  videos = []
  labels = []
  for category in os.listdir(path): #root 폴더의 각 하위폴더들은 클래스이름이기 때문에 이것을 category로 저장
    for video in os.listdir(path+"/"+category): #인자로 받은 경로(path)와 클래스명을 붙여서 각각의 비디오별
      videos.append(path+"/"+category+"/"+video) #비디오의 경로명을 videos에 넣는다.
      labels.append(category) #category명 즉 폴더명이 클래스이므로 label로 넣는다.
  return np.array(videos), np.array(labels) #이것들을 array값으로 반환한다.

#프레임 전처리 함수 preprocess()
def preprocess(frame):
  frame = cv2.resize(frame, (Width, Height))
  # 프레임 픽셀 -1에서 1 범위로 정규화(normalization)
  frame = frame - 127.5 # RGB값 : 255/2 - 0.5
  frame = frame / 127.5
  return frame

#비디오 불러오기 함수 load_video()
def load_video(video_path):
  video_frames = []
  cap = cv2.VideoCapture(video_path) #비디오 경로를 입력하여 cap 인자로 cv2비디오캡처를 실행
  while True:
    ret, frame = cap.read() #비디오의 캡처를 읽어오고
    if ret == True: #실행되는 동안
      video_frames.append(preprocess(frame)) #전처리된 프레임을 video_frames 리스트에 추가
      break
  cap.release()
  video_frames = select_frames(video_frames) #비디오 프레임을 select frames()에 처리시킨다.
  if len(video_frames) != frames:
    print('short_video ',video_path, len(video_frames))
    
  return np.array(video_frames)


def select_frames(video_frames): # video_frames인자는 전처리한 프레임들을 저장한 리스트 형태
  selected_frames = []
  if len(video_frames) > frames: # 전체 비디오 프레임 수가 지정한 프레임 (15)보다 큰 동안
    fn = len(video_frames)//frames # 전체 프레임수를 지정프레임수로 나누고 나머지를 버린 몫을 fn으로 산정 ( //연산자 : 나눈 값에서 소수점을 버린 정수값을 취함)
    # 예를들어 가져온 비디오의 전체프레임이 119프레임이라할때 15로 나누면 7.93333 이고 여기에 나머지를 버리면 7이라는 값이 나옴
    # 즉, 전체프레임 중에서 15프레임 단위로 분할하여 선택적으로 프레임을 가져오기 위해서 설정
    f_num = 0 # 현재 선택된 프레임의 수 
    for f in video_frames: # 각각의 비디오 프레임 값(array배열 값)에 대해서 f로 지정
      if len(selected_frames) < frames: # 만약 선택된 프레임의 수가 전체 할당하고자하는 프레임 수(15)보다 작은 상태라면
        if f_num % fn == 0: # f_num이 fn의 배수가 될 때마다(나누어 떨어질 때마다) 해당 프레임의 위치를 표시
          selected_frames.append(f) # selected_frames에 프레임을 추가
          #즉 fn이 7값이면 7프레임 단위로 끊어서 전체 비디오의 프레임을 가져온다. 7 * 15 = 105, 7 * 16 = 
      f_num += 1 # 선택된 프레임 수 1 증가
  else:
    selected_frames = video_frames #만약 전체 비디오의 프레임 수가 지정한 프레임(15)보다 작은경우 그냥 그프레임 자체를 데이터로 설정
  return selected_frames  #즉, 전체 프레임을 설정한 frames=15라는 값에 의해 15개의 프레임만 선택적으로 균일한 간격을 이루어 선택적으로 취하는 코드


# X, y = create_dataset(videos, encoded_labels, selected_indexes)
def create_dataset(videos, labels, indx):
  X = []
  y = []
  for video, label in zip(videos[indx], labels[indx]):
    X.append(load_video(video))
    y.append(label)

  return np.array(X), np.array(y)



In [12]:
videos, labels = load_video_names('./hmdb51') #path ./hmdb51 내에 있는 각각의 클래스 폴더들의 비디오 이름들을 가져와 videos와 labels로 할당한다.
samples = len(videos)

In [13]:
samples
# print(labels)

1289

In [65]:
#모든 비디오의 label값이 중복으로 나타나는 것을
#np.unique()함수를 이용하여 같은값의 중복을 없앰
classes = len(np.unique(labels)) 


In [66]:
classes

65

In [ ]:
# print(np.unique(labels, return_counts=True))


(array(['나', '만나다', '보다', '비빔밥', '안녕하세요', '얼굴'],
      dtype='<U12'), array([19, 20, 17, 20, 20, 19]))


In [67]:
#비디오가 제대로 불러와졌는지 확인

labels_counts = np.unique(labels, return_counts=True) #각 레이블이 몇 개씩 존재하는지 반환하는 인자 return_counts
print(labels_counts)
print(np.shape(labels_counts))
for l, n in zip(labels_counts[0], labels_counts[1]): #l과 n에 각각 첫번째 배열의 요소와 두번째 배열의 요소를 대응
# l값은 분류 레이블의 이름이 저장되어있고 n값은 해당 레이블의 개수가 저장되어있음
 print(l,n)

(array(['가렵다', '가슴', '고열', '골절', '너무 아파요', '눈',
       '다리', '두드러기생기다', '뒤', '뒤통수', '등', '머리', '목',
       '무릎', '밑에', '발', '발가락', '발목', '발작', '복부',
       '복통', '볼', '부러지다', '붕대', '뼈', '삼키다', '손',
       '손가락', '손목', '심장마비', '쓰러지다', '앞', '어깨',
       '어지러움', '얼굴', '열나다', '옆쪽', '오른쪽', '왼쪽',
       '위에', '응급처리', '이마', '이물질', '인대', '자상',
       '절단', '지혈대', '진통제', '질식', '체온계', '출혈',
       '코', '탈골', '토하다', '팔', '팔꿈치', '피나다',
       '함몰되다', '해독제', '해열제', '허리', '허벅지',
       '호흡곤란', '호흡기', '화상'], dtype='<U15'), array([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 19, 20, 20, 20, 11, 20, 20, 20, 20, 19, 20, 20,
       20, 20, 20, 20, 20, 2

In [16]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

le = LabelEncoder() #LabelEncoder() 객체를 통해 문자로 이루어진 클래스를 숫자 형태로 적용
le.fit(np.unique(labels)) #label의 유니크 값을 le.fit()함수에 넣어 

encoded_labels = le.transform(labels) #fit을 시켜주지 않으면 다음과 같은 에러메세지를 출력 
# This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
# print(encoded_labels) # 결과값 : [63 63 63 ... 2  2  2]
encoded_labels = np.reshape(encoded_labels, (-1,1)) # 하나의 row에 값들을 나열 (수평축의 크기를 1로 해서 1개의 요소가 들어가면 다음 열로 넘어감)
# print(encoded_labels)

encoder = OneHotEncoder()
encoder.fit(encoded_labels) # 나열된 레이블 값들을 원핫 인코더에 적용시키기 위해 .fit()실행
encoded_labels = encoder.transform(encoded_labels)

print(encoded_labels)

  (0, 55)	1.0
  (1, 55)	1.0
  (2, 55)	1.0
  (3, 55)	1.0
  (4, 55)	1.0
  (5, 55)	1.0
  (6, 55)	1.0
  (7, 55)	1.0
  (8, 55)	1.0
  (9, 55)	1.0
  (10, 55)	1.0
  (11, 55)	1.0
  (12, 55)	1.0
  (13, 55)	1.0
  (14, 55)	1.0
  (15, 55)	1.0
  (16, 55)	1.0
  (17, 55)	1.0
  (18, 55)	1.0
  (19, 55)	1.0
  (20, 54)	1.0
  (21, 54)	1.0
  (22, 54)	1.0
  (23, 54)	1.0
  (24, 54)	1.0
  :	:
  (1264, 57)	1.0
  (1265, 57)	1.0
  (1266, 57)	1.0
  (1267, 57)	1.0
  (1268, 57)	1.0
  (1269, 56)	1.0
  (1270, 56)	1.0
  (1271, 56)	1.0
  (1272, 56)	1.0
  (1273, 56)	1.0
  (1274, 56)	1.0
  (1275, 56)	1.0
  (1276, 56)	1.0
  (1277, 56)	1.0
  (1278, 56)	1.0
  (1279, 56)	1.0
  (1280, 56)	1.0
  (1281, 56)	1.0
  (1282, 56)	1.0
  (1283, 56)	1.0
  (1284, 56)	1.0
  (1285, 56)	1.0
  (1286, 56)	1.0
  (1287, 56)	1.0
  (1288, 56)	1.0


In [17]:
# 인코딩 레이블 값을 array값으로 covert
encoded_labels = encoded_labels.toarray()
print(encoded_labels)
#어쨋든 데이터로 들어가야 하는 레이블 값은 다음과 같은 값이어야 함

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
np.shape(encoded_labels) #(영상 개수, 클래스 개수)

(1289, 65)

In [70]:
#케라스 모델 생성
from keras.models import Input, Model
from keras.layers import TimeDistributed, LSTM
from keras.layers import ConvLSTM2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, LeakyReLU, BatchNormalization
from keras.layers import Dense, Flatten, GlobalMaxPooling2D
from keras.layers import MaxPooling3D
from keras.layers import concatenate
from keras.optimizers import Adam

In [71]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def res_block(model, filters):
  '''
  feature extraction 하기 위한 residual block
  1개의 CNN 레이어와 leaky relu action 함수 포함
  '''

  #model = TimeDistributed(Conv2D(filters=filters,
  #kernel_size=3, padding='same'))(model)
  start_block = model
  model = Conv2D(filters=filters, kernel_size=3, padding='same')(model)
  model = BatchNormalization(momentum=0.9)(model)
  model = LeakyReLU(0.2)(model)
  return concatenate([start_block, model])

def create_model():
  '''
  초기모델은 ConvLSTM2D 레이어를 구성
  이를 통해 비디오의 spatial(공간적)특성과 temporal(일시성, 시간성)특성을 캐치
  '''
  input_layer = Input(shape=(frames, Width,Height,3))

  model = ConvLSTM2D(32, 3 , padding='same', return_sequences=False)(input_layer)
  model = BatchNormalization(momentum=0.9)(model)
  model = LeakyReLU(0.2)(model)

  filters = 64 #필터 개수 설정

  for _ in range(6):
    model = res_block(model, filters)
    try:
      model = MaxPooling3D((2,2,2))(model)
    except:
      model = MaxPooling2D((2,2))(model)
    if filters < 512:
      filters *= 2

  #model = TimeDistributed(Conv2D(filters, 3 , padding='same'))(model)
  #model = TimeDistributed(GlobalMaxPooling2D())(model)
  model = Flatten()(model)

  # 출력층
  model = Dense(classes, activation='softmax')(model)
  model = Model(input_layer , model)
  model.compile(optimizer=Adam(learning_rate=1e-4),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
classifier = create_model()

In [ ]:
classifier.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 15, 256, 256 0                                            
__________________________________________________________________________________________________
conv_lst_m2d (ConvLSTM2D)       (None, 256, 256, 32) 40448       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 32) 128         conv_lst_m2d[0][0]               
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 256, 256, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
# 모델 그래프 생성
from keras.utils import plot_model
plot_model(classifier)


In [ ]:
#모델 저장할 경로 설정
model_name = './conv_lstm_2d.h5'

In [ ]:
# pre-train된 가중치를 불러오기
try:
  classifier.load_weights(model_name)
except:
  pass

In [ ]:
batch_size = 4 #batch 사이즈 설정
# total batches

# samples//batch_size = 1289 // 4  = 322 < 1289/4 = 322.25 
if samples//batch_size < samples/batch_size: # 
  batches = (samples//batch_size)+1 # 323
else:
  batches = samples//batch_size 

#model.fit() 대신에 train_on_batch()를 사용하는 이유는 각 에폭마다 가중치의 업데이트가 매번 일어나는 것이 아니라
# 특정 주기에만 업데이트를 시켜주기 위해서 이러한 방법을 사용하는 것이다.
# 또한 각 배치 사이에서 값의 조정이 가능하다.

for e in range(100): #에폭 수 
  index = list(range(samples))
  np.random.shuffle(index) # 인덱스를 저장한 배열을 무작위 순서로 설정
  accuracy = 0
  for batch in range(batches): # 0 ~ 322
    bs = batch*batch_size # MINIMUM = 0 * 4 = 0 ~ MAXIMUM = (322) * 4 =  # 0 ~ 1288
    be = bs+batch_size # MIN = 0 * 4 ~ MAX = 1288 * 4 = 5152
    selected_indexes = index[bs:be] #index [0:1289, 0:5153]

    X, y = create_dataset(videos, encoded_labels, selected_indexes)
    results = classifier.train_on_batch(X,y)
    print('\r', batch, '/', batches, ' : ', results[0], results[1], end='')
    accuracy += results[1]
    if batch%100 == 0:
      classifier.save_weights(model_name)
  print('\r> ',e,', Accuracy = ', accuracy/batches)
  classifier.save_weights(model_name)

>  0 , Accuracy =  0.10344827586206896
>  1 , Accuracy =  0.10344827586206896
>  2 , Accuracy =  0.15804597735404968
>  3 , Accuracy =  0.23275862068965517
>  4 , Accuracy =  0.22413793103448276
>  5 , Accuracy =  0.32183908080232554
>  6 , Accuracy =  0.41091954091499594
>  7 , Accuracy =  0.3620689655172414
>  8 , Accuracy =  0.38218390838853245
>  9 , Accuracy =  0.4712643685012028
>  10 , Accuracy =  0.44252873597473935
>  11 , Accuracy =  0.4856321842506014
>  12 , Accuracy =  0.5747126443632717
>  13 , Accuracy =  0.5431034482758621
 4 / 29  :  0.6163866519927979 0.5

KeyboardInterrupt: ignored